In [1]:
from lark import Lark


In [80]:
expr_test = Lark(r"""
    expression_test : subprogram_body | expression
    
    subprogram_body : [overriding_indicator] subprogram_specification "is" body_section
    body_section : declarative_part # statements "end" NAME

    contract       : classic_procedure_contract | classic_function_contract
                   | spark_2014_subprogram_contract

    classic_procedure_contract : [global_definition] [dependency_relation] [precondition] [postcondition]
    classic_function_contract  : [global_definition] [precondition] [return_definition]
    
    global_definition : "global" mode global_item_list ";" [mode global_item_list ";"]*
    global_item_list  : global_item ["," global_item]*

    dependency_relation : "derives" dependency_clauses
    dependency_clauses : dependency_clause ["&" dependency_clause]* ["&" null_dependency_clause]";"
    dependency_clause : exported_variable_list "from" [extended_imported_item_list]
                      | null_dependency_clause
    null_dependency_clause : NULL "from" imported_item_list

    exported_variable_list : exported_variable ["," exported_variable]*
    extended_imported_item_list : STAR | [STAR ","] imported_item_list
    imported_item_list : imported_item ["," imported_item]*

    precondition : "pre" classic_predicate
    postcondition : "post" classic_predicate

    return_definition : "return" classic_predicate
                      | "return" NAME "=>" classic_predicate

    classic_predicate : expression

    ada_expression : "ada" expression

    name : direct_name | direct_name_tilde   # tilde for SPARK predicates.
         | indexed_component | slice
         | selected_component | selected_component_tilde | attribute_reference
         | character_literal
         | type_conversion | qualified_expression
         | record_update | array_update # record_update and array update for SPARK predicates

    direct_name : NAME
    direct_name_tilde : NAME TILDE
    prefix      : name
    discrete_range : subtype_indication | range  
    range : range_attribute_reference
          | simple_expression ".." simple_expression

    range_constraint : "range" range
    digits_constraint : "digits" simple_expression [range_constraint]
    delta_constraint : "delta" simple_expression [range_constraint]
    index_constraint : "(" discrete_range ["," discrete_range]* ")"

    indexed_component : prefix "(" expression ["," expression]* ")"
    slice : prefix "(" discrete_range ")"

    selected_component : prefix "." selector_name
    selector_name : NAME
                  | character_literal
                  | STRING
    selected_component_tilde : selected_component_tilde

    range_attribute_reference : prefix "'" range_attribute_designator
    range_attribute_designator : "range" ["(" expression ")"]
    attribute_reference : prefix "'" attribute_designator
    attribute_designator : NAME ["(" expression ")"]
                         | DELTA | DIGITS | MOD

    qualified_expression : subtype_mark "'" "(" expression ")"  # | subtype_mark'aggregate
    character_literal : "/."

    type_conversion :subtype_mark "(" expression ")"

    record_update : prefix "[" name "=>" expression [";" name "=>" expression]* "]"
    array_update : prefix "[" index_list "=>" expression [";" index_list "=>" expression]* "]"
    index_list : expression ["," expression]*

    expression : relation [AND relation]*  | relation [AND_THEN relation]*
               | relation [OR relation]*  | relation [OR_ELSE relation]*
               | relation [XOR relation]*
               | relation IMPLIES relation | relation EQUIV relation
               | classic_quantified_expression # IMPLIES, EQUIV and quantified for SPARK predicates

    relation : simple_expression [relational_operator simple_expression]
             | simple_expression [NOT] IN membership_choice_list

    relational_operator : EQ | NEQ | LEQ | GEQ | LT |GT
    
    membership_choice_list : membership_choice ["|" membership_choice]*
    membership_choice : simple_expression | range | subtype_mark

    simple_expression : [unary_adding_operator] term [binary_adding_operator term]*
    term : factor [multiplying_operator factor]*
    factor : primary [EXPONENT primary] | ABS primary | NOT primary
    primary : NUMBER | NULL | STRING  # | aggregate
            | name | parentheized_expression
            | "(" conditional_expression ")" | "(" quantified_expression ")"
            | "(" declare_expression ")"

    unary_adding_operator : PLUS | MINUS
    binary_adding_operator : PLUS | MINUS | ANDPERSAND

    multiplying_operator : TIMES | DIVIDE | MOD | REM
    
    parentheized_expression : "(" expression ")"
    conditional_expression : if_expression | case_expression

    if_expression : "if" expression "then" expression [elsif_part]* [else_part]
    elsif_part : "elsif" expression "then" expression
    else_part : "else" expression

    case_expression : "case" expression "is" case_expression_alternative ["," case_expression_alternative]*

    case_expression_alternative : "when" discrete_choice_list "=>" expression
    discrete_choice_list : discrete_choice ["|" discrete_choice]*

    discrete_choice : expression | subtype_indication | range | OTHERS

    quantified_expression : "for" quantifier loop_parameter_specification "=>" expression
                          # | "for" quantifier iterator_specification "=>" expression

    quantifier : ALL | SOME
    loop_parameter_specification: NAME "in" [REVERSE] discrete_subtype_definition [iterator_filter]
    iterator_filter : "when" expression
    discrete_subtype_definition : subtype_indication | range

    classic_quantified_expression : quantifier_kind quantifier_body
    quantifier_body : NAME "in" subtype_mark ["range" range] "=>" classic_predicate
    quantifier_kind : FOR_ALL | FOR_SOME

    declare_expression : "declare" [declare_item] "begin" expression
    declare_item : object_declaration # | object_renaming_declaration

       aspect_specification : "with" aspect_mark ["=>" aspect_definition] [aspect_list]
    aspect_list : ("," aspect_mark ["=>" aspect_definition])*

    aspect_mark : NAME ["'" CLASS]

    aspect_definition : expression # | aggregate | global_aspect_definition
             
    spark_2014_subprogram_contract : ["with" aspect_clauses]
    aspect_clauses : aspect_clause ["," aspect_clause]*
    
    aspect_clause : [global_aspect] [dependency_aspect] [precondition_aspect] [postcondition_aspect]

    global_aspect : "global" "=>" global_specification
    global_specification : "(" moded_global_list ["," moded_global_list]* ")"
                         | global_list
                         | null_global_specification
    moded_global_list : mode_selector "=>" global_list
    global_list : global_item 
                | "(" global_item ["," global_item]* ")"
    mode_selector     : INPUT | OUTPUT | IN_OUT | PROOF_IN
    null_global_specification : NULL

    dependency_aspect : "depends" "=>" dependency_list
    dependency_list : NULL
                    | "(" dependency_specification ["," dependency_specification]* ")"
    dependency_specification : output_list "=>" [PLUS] input_list
                             | null_dependecy_specification
    null_dependecy_specification : NULL "=>" input_list
    output_list : output
                | "(" output ["," output]*")"
    input_list : input
               | "(" input ["," input]* ")"
               | NULL

    precondition_aspect : "pre" "=>" expression
    postcondition_aspect : "post" "=>" expression
            
    mode : [IN] [OUT]

    declarative_part : [declarative_item]*
    declarative_item : basic_declarative_item | body
    basic_declarative_item : basic_declaration # aspect_clause use_clause

    basic_declaration : type_declaration | subtype_declaration
                      | object_declaration | number_declaration
                      | subprogram_declaration 
                      # | package_declaration | expression_function_declaration
                      # | null_procedure_declaration | expression_function_declaration
                      # | package_declaration | renaming_declaration
                      # | exception_declaration | generic_declaration
                      # | generic_instantiation | abstract_subprogram_declaration

      type_declaration : full_type_declaration
                       | incomplete_type_declaration
                       | private_type_declaration
                       # | private_extension_declaration

    full_type_declaration : "type" NAME [known_discriminant_part] "is" type_definition [aspect_specification] ";"
                          # | task_type_declaration
                          # | protected_type_declaration

    type_definition : enumeration_type_definition | integer_type_definition
                    | real_type_definition | array_type_definition
                    | record_type_definition # | access_type_definition
                    # | derived_type_definition | interface_type_definition

    enumeration_type_definition : "(" enumeration_literals ")" 
    enumeration_literals : enumeration_literal_speififcation ["," enumeration_literal_specification]*
    enumeration_literal_specification : NAME | character_literal

    integer_type_definition : signed_integer_type_definition | modular_type_definition
    signed_integer_type_definition : "range" simple_expression ".." simple_expression
    modular_type_definition : "mod" expression

    real_type_definition : floating_point_definition | fixed_point_definition
    floating_point_definition : "digits" expression [real_range_specification]
    real_range_specification : "range" simple_expression ".." simple_expression

    array_type_definition : unconstrained_array_definition | constrained_array_definition
    unconstrained_array_definition : "array" "(" index_subtypes ")" "of" component_definition
    index_subtypes : index_subtype_definition ["," index_subtype_definition]*
    index_subtype_definition : subtype_mark "range" "<>"
    constrained_array_definition : "array" "(" subtype_definitions ")" "of" component_definition
    subtype_definitions : discrete_subtype_definition ["," discrete_subtype_definition]*
    component_definition : subtype_indication 
                         # [aliased] subtype_indication
                         # | [aliased] access_definition

    record_type_definition : [[ABSTRACT] TAGGED] [LIMITED] record_definition
    record_definition : "record" component_list "end" "record" [NAME]
                      | NULL "record"
    component_list : [component_item]+
                   | [component_item]* variant_part
                   |  NULL ";"
    component_item : component_declaration | aspect_clause
    component_declaration : defining_identifier_list ":" component_definition component_completion ";"
    component_completion : [ASSIGN initialization][aspect_specification]

    subtype_declaration : "subtype" NAME "is" subtype_indication [aspect_specification]";"
    subtype_indication : subtype_mark [constraint]
    # subtype_indication : [null_exclusion] subtype_mark [constraint]
    subtype_mark : name
    
    constraint : scalar_constraint | composite_constraint
    scalar_constraint : range_constraint | digits_constraint | delta_constraint
    composite_constraint : index_constraint # | discriminant_constraint

    object_declaration : defining_identifier_list ":" [[ALIASED] CONSTANT] object_subtype ";"
    object_subtype : subtype_indication object_completion
                   | array_type_definition object_completion
                   # | access_definition object_completion
                   # | single_task_declaration
                   # | single_protected_declaration
    object_completion : [ASSIGN expression] [aspect_specification]
    defining_identifier_list : NAME ["," NAME]*

    number_declaration : defining_identifier_list ":" "constant" ":=" static_expression ";"

    subprogram_declaration : [overriding_indicator] subprogram_specification contract ";"
    subprogram_specification : procedure_specification | function_specification

    procedure_specification : "procedure" defining_program_unit_name parameter_profile
    function_specification : "function" defining_designator parameter_and_result_profile
    defining_designator : defining_program_unit_name # | defining_operator_symbol
    defining_program_unit_name : [parent_unit_name "."] NAME
    parent_unit_name : NAME

    parameter_profile : [formal_part]
    parameter_and_result_profile : [formal_part] "return" [null_exclusion] subtype_mark
                                 # | [formal_part] "return" access_definition
    formal_part : "(" parameter_specification [";" parameter_specification]* ")"
    parameter_specification : defining_identifier_list ":" mode subtype_mark [aspect_specification]
    # parameter_specification ::= defining_identifier_list ":" [aliased] mode 
    #                             [null_exclusion] subtype_mark [:= default_expression]
    #                             [aspect_specification]
    #                           | defining_identifier_list : access_definition [:= default_expression]
    #                             [aspect_specification]
    
    body : proper_body # | body_stub
    proper_body : subprogram_body # | package_body | task_body | protected_body

    proper_body : subprogram_body # | package_body | task_body | protected_body


    exported_variable : NAME
    global_item       : NAME
    imported_item     : NAME
    input             : NAME
    output            : NAME
    procedure_name    : NAME
    type_name         : NAME
    variable_name     : NAME

    ABS      : "abs"
    ABSTRACT : "abstract"
    ALL      : "all"
    AND      : "and"
    AND_THEN : "and" WS "then"
    ANDPERSAND : "&"
    ASSIGN   : ":="
    CLASS    : "class"
    CONSTANT : "constant"
    DELTA    : "delta"
    DIGITS   : "digits"
    DIVIDE   : "/"
    EQ       : "="
    EQUIV    : "<->"
    EXPONENT : "**"
    FOR_ALL  : "for" WS "all"
    FOR_SOME : "for" WS "some"
    GEQ      : ">="
    GLOBAL   : "global"
    GT       : ">"
    IMPLIES  : "->"
    IN       : "in"
    IN_OUT   : "in_out"
    INPUT    : "input"
    LEQ      : "<="
    LIMITED  : "limited"
    LT       : "<"
    MINUS    : "-"
    MOD      : "mod"
    NEQ      : "/="
    NOT      : "not"
    NULL     : "null"
    OR       : "or"
    OR_ELSE  : "or" WS "else"
    OTHERS   : "others"
    OUT      : "out"
    OUTPUT   : "output"
    PLUS     : "+"
    PROOF_IN : "proof_in"
    REM      : "rem"
    REVERSE  : "reverse"
    SOME     : "some"
    STAR     : "*"
    TAGGED   : "tagged"
    TILDE    : "~"
    TIMES    : "*"
    XOR      : "xor"
    
    %import common.CNAME -> NAME
    %import common.ESCAPED_STRING -> STRING
    %import common.INT -> NUMBER
    %import common.WS
    %ignore WS

    ANNO : "--#"
    %ignore ANNO

    """, start='expression_test', lexer='basic')


GrammarError: Rule 'proper_body' defined more than once

In [49]:
text='''procedure P1 (a : in integer; b : in out integer; c : natural)'''
print (text)
expr_test.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)


Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'classic_subprogram_contract'), [None, None, None, None])])])])

In [50]:
print( _.pretty() )

expression_test
  procedure_body
    procedure_specification
      procedure_name	P1
      parameter_list
        parameter
          variable_name	a
          mode
            in
            None
          type_name	integer
        parameter
          variable_name	b
          mode
            in
            out
          type_name	integer
        parameter
          variable_name	c
          mode
            None
            None
          type_name	natural
    contract
      classic_subprogram_contract
        None
        None
        None
        None



In [51]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;'''
print (text)
expr_test.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;


Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'classic_subprogram_contract'), [Tree(Token('RULE', 'global_definition')

In [52]:
print( _.pretty() )

expression_test
  procedure_body
    procedure_specification
      procedure_name	P1
      parameter_list
        parameter
          variable_name	a
          mode
            in
            None
          type_name	integer
        parameter
          variable_name	b
          mode
            in
            out
          type_name	integer
        parameter
          variable_name	c
          mode
            None
            None
          type_name	natural
    contract
      classic_subprogram_contract
        global_definition
          mode
            in
            None
          global_item_list
            global_item	g
            global_item	h
            global_item	j
          mode
            in
            out
          global_item_list
            global_item	r
            global_item	s
            global_item	t
          mode
            None
            out
          global_item_list
            global_item	x
            global_item	y
            global_item	z
       

In [53]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          with global => (input => (g, h, j), 
                     in_out => (r, s, j), 
                     output => (x, y, z))'''
print (text)
expr_test.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          with global => (input => (g, h, j), 
                     in_out => (r, s, j), 
                     output => (x, y, z))


Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'spark_2014_subprogram_contract'), [Tree(Token('RULE', 'aspect_clauses')

In [54]:
print( _.pretty() )

expression_test
  procedure_body
    procedure_specification
      procedure_name	P1
      parameter_list
        parameter
          variable_name	a
          mode
            in
            None
          type_name	integer
        parameter
          variable_name	b
          mode
            in
            out
          type_name	integer
        parameter
          variable_name	c
          mode
            None
            None
          type_name	natural
    contract
      spark_2014_subprogram_contract
        aspect_clauses
          aspect_clause
            global_aspect
              global_specification
                moded_global_list
                  mode_selector	input
                  global_list
                    global_item	g
                    global_item	h
                    global_item	j
                moded_global_list
                  mode_selector	in_out
                  global_list
                    global_item	r
                    global_item	s
   

In [55]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;
          --# derives r, x, y from g, h, s &
                      s from *             &
                      t from *, y, z;'''
print (text)
expr_test.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;
          --# derives r, x, y from g, h, s &
                      s from *             &
                      t from *, y, z;


Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'classic_subprogram_contract'), [Tree(Token('RULE', 'global_definition')

In [56]:
print( _.pretty() )

expression_test
  procedure_body
    procedure_specification
      procedure_name	P1
      parameter_list
        parameter
          variable_name	a
          mode
            in
            None
          type_name	integer
        parameter
          variable_name	b
          mode
            in
            out
          type_name	integer
        parameter
          variable_name	c
          mode
            None
            None
          type_name	natural
    contract
      classic_subprogram_contract
        global_definition
          mode
            in
            None
          global_item_list
            global_item	g
            global_item	h
            global_item	j
          mode
            in
            out
          global_item_list
            global_item	r
            global_item	s
            global_item	t
          mode
            None
            out
          global_item_list
            global_item	x
            global_item	y
            global_item	z
       

In [57]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          with
             global => (input => (g, h, j),
                       in_out => (r, s, t),
                       output =>  (x, y, z)),
             depends => ((r, x, y) => (g, h, s),
                         s => + null,
                         t => + (y, z))'''
print (text)
expr_test.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          with
             global => (input => (g, h, j),
                       in_out => (r, s, t),
                       output =>  (x, y, z)),
             depends => ((r, x, y) => (g, h, s),
                         s => + null,
                         t => + (y, z))


Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'spark_2014_subprogram_contract'), [Tree(Token('RULE', 'aspect_clauses')

In [58]:
print( _.pretty() )

expression_test
  procedure_body
    procedure_specification
      procedure_name	P1
      parameter_list
        parameter
          variable_name	a
          mode
            in
            None
          type_name	integer
        parameter
          variable_name	b
          mode
            in
            out
          type_name	integer
        parameter
          variable_name	c
          mode
            None
            None
          type_name	natural
    contract
      spark_2014_subprogram_contract
        aspect_clauses
          aspect_clause
            global_aspect
              global_specification
                moded_global_list
                  mode_selector	input
                  global_list
                    global_item	g
                    global_item	h
                    global_item	j
                moded_global_list
                  mode_selector	in_out
                  global_list
                    global_item	r
                    global_item	s
   

In [59]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;
          --# derives r, x, y from g, h, s &
                      s from *             &
                      t from *, y, z;
          --# pre "g > 0"
          --# post "x = g + h"'''
print (text)
expr_test.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;
          --# derives r, x, y from g, h, s &
                      s from *             &
                      t from *, y, z;
          --# pre "g > 0"
          --# post "x = g + h"


Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'classic_subprogram_contract'), [Tree(Token('RULE', 'global_definition')

In [60]:
print( _.pretty() )

expression_test
  procedure_body
    procedure_specification
      procedure_name	P1
      parameter_list
        parameter
          variable_name	a
          mode
            in
            None
          type_name	integer
        parameter
          variable_name	b
          mode
            in
            out
          type_name	integer
        parameter
          variable_name	c
          mode
            None
            None
          type_name	natural
    contract
      classic_subprogram_contract
        global_definition
          mode
            in
            None
          global_item_list
            global_item	g
            global_item	h
            global_item	j
          mode
            in
            out
          global_item_list
            global_item	r
            global_item	s
            global_item	t
          mode
            None
            out
          global_item_list
            global_item	x
            global_item	y
            global_item	z
       

In [61]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          with
             global => (input => (g, h, j),
                       in_out => (r, s, t),
                       output =>  (x, y, z)),
             depends => ((r, x, y) => (g, h, s),
                         s => + null,
                         t => + (y, z)),
             pre     => g > 0,
             post    => x = g + h'''
print (text)
expr_test.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          with
             global => (input => (g, h, j),
                       in_out => (r, s, t),
                       output =>  (x, y, z)),
             depends => ((r, x, y) => (g, h, s),
                         s => + null,
                         t => + (y, z)),
             pre     => g > 0,
             post    => x = g + h


Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'spark_2014_subprogram_contract'), [Tree(Token('RULE', 'aspect_clauses')

In [62]:
print( _.pretty() )

expression_test
  procedure_body
    procedure_specification
      procedure_name	P1
      parameter_list
        parameter
          variable_name	a
          mode
            in
            None
          type_name	integer
        parameter
          variable_name	b
          mode
            in
            out
          type_name	integer
        parameter
          variable_name	c
          mode
            None
            None
          type_name	natural
    contract
      spark_2014_subprogram_contract
        aspect_clauses
          aspect_clause
            global_aspect
              global_specification
                moded_global_list
                  mode_selector	input
                  global_list
                    global_item	g
                    global_item	h
                    global_item	j
                moded_global_list
                  mode_selector	in_out
                  global_list
                    global_item	r
                    global_item	s
   

In [64]:
expr_test.parse(" x > y and z = 0 and (not w = 3)")

Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'expression'), [Tree(Token('RULE', 'relation'), [Tree(Token('RULE', 'simple_expression'), [None, Tree(Token('RULE', 'term'), [Tree(Token('RULE', 'factor'), [Tree(Token('RULE', 'primary'), [Tree(Token('RULE', 'name'), [Tree(Token('RULE', 'direct_name'), [Token('NAME', 'x')])])]), None, None]), None, None]), None, None]), Tree(Token('RULE', 'relational_operator'), [Token('GT', '>')]), Tree(Token('RULE', 'simple_expression'), [None, Tree(Token('RULE', 'term'), [Tree(Token('RULE', 'factor'), [Tree(Token('RULE', 'primary'), [Tree(Token('RULE', 'name'), [Tree(Token('RULE', 'direct_name'), [Token('NAME', 'y')])])]), None, None]), None, None]), None, None])]), Token('AND', 'and'), Tree(Token('RULE', 'relation'), [Tree(Token('RULE', 'simple_expression'), [None, Tree(Token('RULE', 'term'), [Tree(Token('RULE', 'factor'), [Tree(Token('RULE', 'primary'), [Tree(Token('RULE', 'name'), [Tree(Token('RULE', 'direct_name'), [Token('NAME', 'z')])]

In [44]:
print( _.pretty() )

expression_test
  ada_expression
    expression
      relation
        simple_expression
          None
          term
            factor
              primary
                name
                  direct_name	x
              None
              None
            None
            None
          None
          None
        relational_operator	>
        simple_expression
          None
          term
            factor
              primary
                name
                  direct_name	y
              None
              None
            None
            None
          None
          None
      and
      relation
        simple_expression
          None
          term
            factor
              primary
                name
                  direct_name	z
              None
              None
            None
            None
          None
          None
        relational_operator	=
        simple_expression
          None
          term
            factor
              primary

In [65]:
expr_test.parse("x > y and z = 0 and (not w = 3)")

Tree(Token('RULE', 'expression_test'), [Tree(Token('RULE', 'expression'), [Tree(Token('RULE', 'relation'), [Tree(Token('RULE', 'simple_expression'), [None, Tree(Token('RULE', 'term'), [Tree(Token('RULE', 'factor'), [Tree(Token('RULE', 'primary'), [Tree(Token('RULE', 'name'), [Tree(Token('RULE', 'direct_name'), [Token('NAME', 'x')])])]), None, None]), None, None]), None, None]), Tree(Token('RULE', 'relational_operator'), [Token('GT', '>')]), Tree(Token('RULE', 'simple_expression'), [None, Tree(Token('RULE', 'term'), [Tree(Token('RULE', 'factor'), [Tree(Token('RULE', 'primary'), [Tree(Token('RULE', 'name'), [Tree(Token('RULE', 'direct_name'), [Token('NAME', 'y')])])]), None, None]), None, None]), None, None])]), Token('AND', 'and'), Tree(Token('RULE', 'relation'), [Tree(Token('RULE', 'simple_expression'), [None, Tree(Token('RULE', 'term'), [Tree(Token('RULE', 'factor'), [Tree(Token('RULE', 'primary'), [Tree(Token('RULE', 'name'), [Tree(Token('RULE', 'direct_name'), [Token('NAME', 'z')])]